# 逻辑回归解决网络数据分类

In [1]:
import numpy as np
import pandas as pd

## Step1. 导入数据

In [8]:
## TODO：选择自己需要的数据数量

path = 'data/project1_data.csv'
pd_data = pd.read_csv(path,nrows=10000)
pd_data.head()

,Flow.ID,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,...,Active.Std,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName
0,172.19.1.46-10.200.7.7-52422-3128-6,172.19.1.46,52422,10.200.7.7,3128,6,26/04/201711:11:17,45523,22,55,...,0.0,0,0,0.0,0.0,0,0,BENIGN,131,HTTP_PROXY
1,172.19.1.46-10.200.7.7-52422-3128-6,10.200.7.7,3128,172.19.1.46,52422,6,26/04/201711:11:17,1,2,0,...,0.0,0,0,0.0,0.0,0,0,BENIGN,131,HTTP_PROXY
2,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,1,3,0,...,0.0,0,0,0.0,0.0,0,0,BENIGN,7,HTTP
3,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,217,1,3,...,0.0,0,0,0.0,0.0,0,0,BENIGN,7,HTTP
4,192.168.72.43-10.200.7.7-55961-3128-6,192.168.72.43,55961,10.200.7.7,3128,6,26/04/201711:11:17,78068,5,0,...,0.0,0,0,0.0,0.0,0,0,BENIGN,131,HTTP_PROXY


## Step.2 数据处理

### 2.1 数据丢弃

In [9]:
pd_data.keys()

Index(['Flow.ID', 'Source.IP', 'Source.Port', 'Destination.IP',
       'Destination.Port', 'Protocol', 'Timestamp', 'Flow.Duration',
       'Total.Fwd.Packets', 'Total.Backward.Packets',
       'Total.Length.of.Fwd.Packets', 'Total.Length.of.Bwd.Packets',
       'Fwd.Packet.Length.Max', 'Fwd.Packet.Length.Min',
       'Fwd.Packet.Length.Mean', 'Fwd.Packet.Length.Std',
       'Bwd.Packet.Length.Max', 'Bwd.Packet.Length.Min',
       'Bwd.Packet.Length.Mean', 'Bwd.Packet.Length.Std', 'Flow.Bytes.s',
       'Flow.Packets.s', 'Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max',
       'Flow.IAT.Min', 'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std',
       'Fwd.IAT.Max', 'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean',
       'Bwd.IAT.Std', 'Bwd.IAT.Max', 'Bwd.IAT.Min', 'Fwd.PSH.Flags',
       'Bwd.PSH.Flags', 'Fwd.URG.Flags', 'Bwd.URG.Flags', 'Fwd.Header.Length',
       'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s',
       'Min.Packet.Length', 'Max.Packet.Length', 'Packet.Length.Mean',
  

In [10]:
## TODO: 选择自己想要丢弃的列

drop_ids = ['Flow.ID','Source.IP', 'Source.Port','Destination.IP','Destination.Port','Timestamp','Label','L7Protocol']

In [11]:
datas = pd_data.drop(drop_ids, axis=1)
datas.shape

(10000, 79)

### 2.2 制作标签

In [12]:
def create_search(datas):
    protocol_to_num = {}
    num_to_protocol = {}
    label_data = datas.ProtocolName.to_numpy()
    init_num = 0
    for i in range(label_data.shape[0]):
        if label_data[i] not in protocol_to_num.keys():
            protocol_to_num[label_data[i]] = init_num
            num_to_protocol[init_num] = label_data[i]
            init_num += 1
    return protocol_to_num, num_to_protocol

P_N, N_P = create_search(datas)

In [13]:
P_N

{'HTTP_PROXY': 0,
 'HTTP': 1,
 'HTTP_CONNECT': 2,
 'SSL': 3,
 'GOOGLE': 4,
 'YOUTUBE': 5,
 'FACEBOOK': 6,
 'CONTENT_FLASH': 7,
 'DROPBOX': 8,
 'WINDOWS_UPDATE': 9,
 'AMAZON': 10,
 'MICROSOFT': 11,
 'TOR': 12,
 'GMAIL': 13,
 'YAHOO': 14,
 'MSN': 15,
 'SSL_NO_CERT': 16,
 'SKYPE': 17,
 'MS_ONE_DRIVE': 18,
 'MSSQL': 19,
 'TWITTER': 20,
 'APPLE_ICLOUD': 21,
 'CLOUDFLARE': 22}

In [14]:
N_P

{0: 'HTTP_PROXY',
 1: 'HTTP',
 2: 'HTTP_CONNECT',
 3: 'SSL',
 4: 'GOOGLE',
 5: 'YOUTUBE',
 6: 'FACEBOOK',
 7: 'CONTENT_FLASH',
 8: 'DROPBOX',
 9: 'WINDOWS_UPDATE',
 10: 'AMAZON',
 11: 'MICROSOFT',
 12: 'TOR',
 13: 'GMAIL',
 14: 'YAHOO',
 15: 'MSN',
 16: 'SSL_NO_CERT',
 17: 'SKYPE',
 18: 'MS_ONE_DRIVE',
 19: 'MSSQL',
 20: 'TWITTER',
 21: 'APPLE_ICLOUD',
 22: 'CLOUDFLARE'}

In [15]:
datas = datas.to_numpy()
for i in range(datas.shape[0]):
    datas[i,-1] = P_N[datas[i,-1]]

### 2.3 打乱数据

In [16]:
print(datas[0:3,0:3])
np.random.shuffle(datas)
print('*'*20)
print(datas[0:3,0:3])
datas = datas.astype(np.float32)

[[6 45523 22]
 [6 1 2]
 [6 1 3]]
********************
[[6 2857279 928]
 [6 44025 2]
 [6 248 2]]


## Checkpoint

In [17]:
np.save('data/project2.npy', datas)

## Step.3 数据分类

In [18]:
import numpy as np
import pandas as pd
from sklearn import linear_model

np.set_printoptions(suppress=True)

Datas = np.load('data/project2.npy')

## TODO：尝试不同的分类比例

ratio = 0.8
split_num = int(Datas.shape[0] * ratio)
train_x = Datas[0:split_num, 0:-1]
train_y = Datas[0:split_num, -1]
test_x = Datas[split_num:, 0:-1]
test_y = Datas[split_num:, -1]

## Step.4 建立模型并训练

In [23]:
## TODO: 尝试不同训练次数，找到你的最佳得分

model = linear_model.LogisticRegression(max_iter=5000)
model.fit(train_x,train_y)
print("训练得分是：{}".format(model.score(train_x, train_y)))
print("测试得分是：{}".format(model.score(test_x,test_y)))

训练得分是：0.55925
测试得分是：0.5375


D:\dev_tools\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
